In [1]:
import sys
import yt.units as u
import numpy
import numpy as np
import matplotlib.pyplot as plt

from dengo.chemical_network import \
    ChemicalNetwork, \
    reaction_registry, \
    cooling_registry, species_registry
import dengo.primordial_rates, dengo.primordial_cooling
from dengo.chemistry_constants import tiny, kboltz, mh


 using cli
 using CLI for selections


In [6]:
import sys
sys.path.append("../examples/cvdls_examples/9species_cooling") 
sys.path.append("../examples/cvspils_examples/9species_cooling") 
sys.path.append("../examples/be_chem_solve_examples/9species_cooling/") 

from generate_dls_cooling_network import run_cvdls
from generate_cvspils_cooling_network import run_cvspils
from generate_bechem_cooling_network import run_bechem

In [7]:
def Init_values(temperature, density, n_species = 9, cooling=True):
    """ Create a initial value dictionary,
        for a given temperature, density, number of species
    Args:
        temperature -- in Kelvin
        density     -- in amu / cm**3
        n_species   -- number of species (6/9)
        cooling

    Returns:
        init_values: initial value dictionary with
                     self-consistent energy/ electron density
        primordial : chemical_network classes

    """

    # initialize and setup the network
    dengo.primordial_rates.setup_primordial()
    primordial = ChemicalNetwork()
    primordial.threebody = 4

    if n_species == 9:
        for i in range(22):
            try:
                primordial.add_reaction("k{0:02d}".format(i+1))
            except:
                pass
    else:
        for i in range(6):
            try:
                primordial.add_reaction("k{0:02d}".format(i+1))
            except:
                pass

    # the temperature array required to interpolates the rates
    primordial.init_temperature((1e0, 1e5))

    tiny = 1.0e-20

    # init_array are is in fractional abundances
    init_values = dict()
    density = numpy.array([density])
    if n_species == 6:
        # 6-species model
        init_values["He_1"]    = density * (1.0 - 0.76)
        init_values["He_2"]    = np.array([tiny])
        init_values["He_3"]    = np.array([tiny])
        init_values["H_1"]     = density *  (0.76)
        init_values['H_2']     = np.array([tiny]) 
    else:
        # 9-species model
        init_values["He_1"]    = density * (1.0 - 0.76) 
        init_values["He_2"]    = density *np.array([tiny])
        init_values["He_3"]    = density *np.array([tiny])
        init_values["H_1"]     = density *  (0.76) / 2.
        init_values['H_2']     = density *np.array([tiny])

        init_values["H_m0"]    = density *np.array([tiny])
        init_values["H2_1"]    = density *  (0.76) / 2.
        init_values["H2_2"]    = density *np.array([tiny])

    # now everything in mass density
    init_values['de'] = primordial.calculate_free_electrons(init_values)
    # one signle value: again mass density
    init_values['density'] = primordial.calculate_total_density(init_values)

    num_den = {}
    for sp in primordial.required_species:
        try:
            num_den[sp.name] = init_values[sp.name]/ sp.weight
            print(sp.name, sp.weight)
        except:
            pass
    print( init_values['density'])
    print(num_den)
    # set up initial temperatures values used to define ge
    init_values['T'] = numpy.array([temperature])

    # calculate gammaH2
    x = 6100.0/temperature
    expx = numpy.exp(x)
    gammaH2 = 2.0 / (5.0 + 2.0*x*x*expx / (expx - 1 )**2.0 ) + 1
    print(gammaH2, gammaH2-7./5.)
    gamma_factor = primordial.gamma_factor().subs(num_den).subs({'gammaH2': gammaH2 , 'gamma': 5./3.,'T': temperature })

    ge  = ((temperature *  kboltz) *gamma_factor
                         / (init_values['density'] * mh  ))

    T = init_values['density']*ge*mh / kboltz / gamma_factor
    print("difference in temperature:", T - temperature)
    init_values['ge'] = numpy.array( [numpy.float64(ge)] )

    if cooling:
        for cooling_action in cooling_registry:
            k = cooling_registry[cooling_action]
            if (k.species).issubset( primordial.required_species ):
                if k.name != "cie_cooling":                    
                    print("adding:", k.name, k.equation)
                    primordial.add_cooling(cooling_action)
                    print('---------------------------')
    return init_values, primordial

In [8]:
init, primordial = Init_values(initial_temperature, 
                               density, n_species = 9, 
                               cooling=False)

print(primordial.gamma_factor())
primordial.temperature_calculation()


NameError: name 'initial_temperature' is not defined

In [11]:
density = 1e10
initial_temperature = 2000.0
final_time = 1e10

final_time_Myr = u.second.in_units('Myr').v * final_time

init, primordial = Init_values(initial_temperature, 
                               density, n_species = 9, 
                               cooling=False)



# # run our solver
rv, cvdls_data, run_time_cvdls = run_cvdls( initial_temperature, 
                                          density, final_time, 
                                          init=init, primordial=primordial, max_iter = 1)

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2
Adding reaction: k07 : 1*H_1 + 1*de => 1*H_m0
Adding reaction: k08 : 1*H_m0 + 1*H_1 => 1*H2_1 + 1*de
Adding reaction: k09 : 1*H_1 + 1*H_2 => 1*H2_2
Adding reaction: k10 : 1*H2_2 + 1*H_1 => 1*H2_1 + 1*H_2
Adding reaction: k11 : 1*H2_1 + 1*H_2 => 1*H2_2 + 1*H_1
Adding reaction: k12 : 1*H2_1 + 1*de => 2*H_1 + 1*de
Adding reaction: k13 : 1*H2_1 + 1*H_1 => 3*H_1
Adding reaction: k14 : 1*H_m0 + 1*de => 1*H_1 + 2*de
Adding reaction: k15 : 1*H_m0 + 1*H_1 => 2*H_1 + 1*de
Adding reaction: k16 : 1*H_m0 + 1*H_2 => 2*H_1
Adding reaction: k17 : 1*H_m0 + 1*H_2 => 1*H2_2 + 1*de
Adding reaction: k18 : 1*H2_2 + 1*de => 2*H_1
Adding reaction: k19 : 1*H2_2 + 1*H_m0 => 1*H_1 + 1*H2_1
Adding reaction: k21 : 2*H_1 

/home/kwoksun2/anaconda2/lib/python2.7/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in multiply
  """
/home/kwoksun2/anaconda2/lib/python2.7/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in multiply
  """
/home/kwoksun2/anaconda2/lib/python2.7/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in multiply
  """
/home/kwoksun2/anaconda2/lib/python2.7/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in multiply
  """


initial  temperature: 2.000E+03
Successful iteration[    0]: (1.000e+10) 1.000e+10 / 1.000e+10
End in 1 iterations: 1.00000e+10 / 1.00000e+10 (0.00000e+00)


In [12]:
cvdls_data

{'H2_1': array([[ 85446860.41438703]]),
 'H2_2': array([[  4.54468306e-18]]),
 'H_1': array([[  7.36923764e+09]]),
 'H_2': array([[  1.48818378e-10]]),
 'H_m0': array([[  1.55146899e-16]]),
 'He_1': array([[  5.99609954e+08]]),
 'He_2': array([[  2.49837481e-11]]),
 'He_3': array([[  2.49837481e-11]]),
 'T': array([[ 1994.79708564]]),
 'de': array([[  2.23769471e-10]]),
 'dt': array([  1.00000000e+10]),
 'ge': array([[  2.01334255e+11]]),
 'successful': array([ True], dtype=bool),
 't': array([  1.00000000e+10])}